In [6]:
import urllib
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementClickInterceptedException
import matplotlib.pyplot as plt
import numpy as np
import dbm 
import pickle


In [4]:
# HELPER FUNCTIONS TO SCRAPE ODDSPORTAL ODDS

def fi(a):
    try:
        driver.find_element_by_xpath(a).text
    except:
        return False
    
def ffi(a):
    if(fi(a)!= False):
        return driver.find_element_by_xpath(a).text

def ell(i,j):
    return ffi('//*[@id="odds-data-table"]/div[1]/table/tbody/tr['+str(i)+']/td['+str(j)+']/div')

def f(i,x):
    try:
        x[ell(i,1)] = [ell(i,2),ell(i,3),ell(i,4)]
    except:
        print('')

def operation():
    time.sleep(1)
    x = dict()
    for i in range(1,12):
        f(i,x)
    try:
        x["result"] = driver.find_element_by_xpath('//*[@id="event-status"]/p/strong').text
    except:
        x["result"] = "Postponed"
    
    time.sleep(2)
    driver.back()
    return x

def is_game(j):
    try:
        driver.find_element_by_xpath('//*[@id="table-matches"]/table/tbody/tr['+str(j)+']/td[2]/a').text
    except:
        return False      
    

#ACTUAL SCRAPER

def op_scrape(link,m,n):
    driver.get(link)
    day_dict = dict()
    for i in range(m,n):
        time.sleep(1)
        if(is_game(i) != False):
            game = driver.find_element_by_xpath('//*[@id="table-matches"]/table/tbody/tr['+str(i)+']/td[2]/a').text
            time.sleep(1)
            try:
                driver.find_element_by_xpath('//*[@id="table-matches"]/table/tbody/tr['+str(i)+']/td[2]/a').click()
                    
            except ElementClickInterceptedException:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                driver.find_element_by_xpath('//*[@id="table-matches"]/table/tbody/tr['+str(i)+']/td[2]/a').click()

            odds = operation()
            day_dict[game] = odds
    
    return day_dict   

#FUNCTIONS TO STORE RESULTS IN DB

def pickle(date,data):
    db = dbm.open('GI_OP', 'c')

    pchdr = pickle.dumps(data)
    db[date] = pchdr

    db.close()
    
def unpickle(date,file):
    db = dbm.open(file,'c')
    pchdr = db[date]
    db.close()
    return pickle.loads(pchdr)

def merge(dict1, dict2):
    res = {**dict1, **dict2}
    return res

#FUNCTION TO READ IN GOALIMPACT ODDS

def gi_comp(file):
    gi_odds = open(file,"r")
    gi_odds = gi_odds.read()
    gi_odds = gi_odds.splitlines()
    games = list()
    for i in range(1,len(gi_odds)):
        if len(gi_odds[i]) > 22:
            games.append(i)
    
    output = dict()
    for j in games:
        inner_dict = dict()
        home = [gi_odds[j+2][4:8],gi_odds[j+2][11:15],gi_odds[j+2][18:22]]
        away = [gi_odds[j+3][4:8],gi_odds[j+3][11:15],gi_odds[j+3][18:22]]
        inner_dict["Home"] = home
        inner_dict["Away"] = away
        output[gi_odds[j][7:]] = inner_dict
    return output


#HELPFER FUNCTION TO MATCH FIXTURES DESCRIBED ON ODDSPORTAL TO GOALIMPACT DESCRIPTIONS

def levenshtein(token1, token2):
    distances = numpy.zeros((len(token1) + 1, len(token2) + 1))

    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1

    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2
        
    a = 0
    b = 0
    c = 0
    
    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                
                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1

    return distances[len(token1)][len(token2)]


def printDistances(distances, token1Length, token2Length):
    for t1 in range(token1Length + 1):
        for t2 in range(token2Length + 1):
            print(int(distances[t1][t2]), end=" ")
        print()
        

# ODDS COMPARISON FOR SINGLE GAME
    
def odds_compare(game,gi_file,day,home,neutral):
    key_dist = 1000 
    key_name = ""
    for key in day.keys():
        pchdr = levenshtein(game,key)
        if pchdr < key_dist:
            key_dist = pchdr
            key_name = key

            
            
    if " Pinnacle  " in day[key_name].keys():     
    
        edge = [float(day[key_name][" Pinnacle  "][0])*(home*float(gi_file[game]["Home"][0])+neutral*float(gi_file[game]["Away"][0])),float(day[key_name][" Pinnacle  "][1])*(home*float(gi_file[game]["Home"][1])+neutral*float(gi_file[game]["Away"][1])),float(day[key_name][" Pinnacle  "][2])*(home*float(gi_file[game]["Home"][2])+neutral*float(gi_file[game]["Away"][2]))]
        if float(max(edge)) > 110:
            bet = edge.index(max(edge))
        else:
            print("Outer space")
            bet = "No"
        
        if bet != "No":
            if int(day[key_name]["result"][0]) < int(day[key_name]["result"][2]):
                result = 2
            elif int(day[key_name]["result"][0]) == int(day[key_name]["result"][2]):
                result = 1
            else:
                result = 0
        
            if result == bet:
                returnz = float(day[key_name][" Pinnacle  "][result])-1
            else:   
                returnz = -1
            
        elif bet == "No":
            result = "-"
            returnz = 0
    
        return [key_name,game,day[key_name][" Pinnacle  "],gi_file[game]["Home"],edge,bet, result, returnz]
    else:
        return "Pinnacle offers no odds"


# EVALUATION WITH OPTION OF DATE SPECIFICATION (DAYS AND KEYS GIVEN)
    
def gi_dayres(link,game,gi_file,day,home,neutral):
    key_dist = 1000 
    key_name = ""
    for key in day.keys():
        pchdr = levenshtein(game,key)
        if pchdr < key_dist:
            key_dist = pchdr
            key_name = key
    print(key_name)
    
    while True:
        try:
            
            driver.get(link)
            time.sleep(1)
            driver.find_element_by_xpath('//*[@id="table-matches"]/table/tbody/tr['+str(day[key_name])+']/td[2]/a').click()
            print(driver.current_url)
            if driver.current_url != link:
                op_odds = operation()
                break
        except NoSuchElementException:
            print("error")
        except ElementClickInterceptedException:
            print("error")
            
            
    if " Pinnacle  " in op_odds.keys():     
    
        edge = [float(op_odds[" Pinnacle  "][0])*(home*float(gi_file[game]["Home"][0])+neutral*float(gi_file[game]["Away"][0])),float(op_odds[" Pinnacle  "][1])*(home*float(gi_file[game]["Home"][1])+neutral*float(gi_file[game]["Away"][1])),float(op_odds[" Pinnacle  "][2])*(home*float(gi_file[game]["Home"][2])+neutral*float(gi_file[game]["Away"][2]))]
        if float(max(edge)) > 110:
            bet = edge.index(max(edge))
        else:
            print("Outer space")
            bet = "No"
        
        if bet != "No":
            if int(op_odds["result"][0]) < int(op_odds["result"][2]):
                result = 2
            elif int(op_odds["result"][0]) == int(op_odds["result"][2]):
                result = 1
            else:
                result = 0
        
            if result == bet:
                returnz = float(op_odds[" Pinnacle  "][result])-1
            else:   
                returnz = -1
            
        elif bet == "No":
            result = "-"
            returnz = 0
    
        return [key_name,game,op_odds[" Pinnacle  "],gi_file[game]["Home"],edge,bet, result, returnz]
    else:
        return "Pinnacle offers no odds"

    
def games_op(link,m,n):
    driver = webdriver.Chrome(executable_path = "/Users/pietervantol/Documents/chromedriver")
    driver.get(link)
    games_dict = dict()
    for i in range(m,n):
        try: 
            games_dict[driver.find_element_by_xpath('//*[@id="table-matches"]/table/tbody/tr['+str(i)+']/td[2]/a').text] = i
        except:
            continue
    return games_dict

In [35]:
#Examples

#1 Dumping results 
db = dbm.open('GI_OP', 'c')

pchdr = pickle.dumps(strat_res)
db["Jul13_24oddscomp"] = pchdr

db.close()

#2 Scraping Pinnacle Odds
Jul16_20 = op_scrape("https://www.oddsportal.com/matches/soccer/20200716/",1,122)

#3 comparing odds
odds_compare("Sporting Kansas City vs. Minnesota United FC",gi_file,Jul_20,1,0)

#4
strat_res_mix = list()
for keys in gi_20:
    strat_res_mix.append(odds_compare(keys,gi_file,Jul20,1,0))

    
returned_mix = 0
for ind in strat_res_mix:
    if ind != "Pinnacle offers no odds":
        print(ind[7])
        returned_mix = returned_mix + ind[7]

print(returned/len(strat_res))
print(returned)
print(len(strat_res))

returned = 0
for ind in strat_res:
    if ind != "Pinnacle offers no odds":
        print(ind[7])
        returned = returned + ind[7]
        

In [8]:
# Example 27th Jul 2020: Evaluation of strategy: Bet when GoalImpactOdds show value 

jul27_game_n= games_op("https://www.oddsportal.com/matches/soccer/20200726/",1,353)
gi_jul27 = gi_comp("GI_Jul27_20.txt")
res_27_7 = list()
driver = webdriver.Chrome(executable_path = "/Users/pietervantol/Documents/chromedriver")
driver.get("https://www.oddsportal.com/matches/soccer/20200726/")
for keys in gi_jul27:
    res_27_7.append(gi_dayres("https://www.oddsportal.com/matches/soccer/20200726/",keys,gi_jul27,jul27_game_n,1,0))

returned_27jul = 0
for ind in res_27_7:
    if ind != "Pinnacle offers no odds":
        returned_27jul = returned_27jul + ind[7]
        print(ind[1],ind[7])

print(returned_27jul)

San Jose Earthquakes vs. Real Salt Lake -1
AC Pisa 1909 vs. Ascoli Picchio FC 1898 0
FC Wil 1900 vs. Grasshopper Club Zürich 4.39
AS Cittadella vs. Venezia FC 1.35
SS Juve Stabia vs. US Cremonese -1
Empoli FC vs. Cosenza Calcio 0
Perugia Calcio vs. ASD Trapani Calcio -1
FC Crotone vs. Frosinone Calcio 2.09
Pordenone Calcio vs. US Salernitana 1919 0
ASD Spezia Calcio 2008 vs. ACD Virtus Entella 0
Cardiff City FC vs. Fulham FC -1
Toronto FC vs. New York City FC -1
2.83
